This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [ ]:
from itertools import product

import numpy as np
from pydrake.all import (AddMultibodyPlantSceneGraph, ContactVisualizer,
                         DiagramBuilder, ExternallyAppliedSpatialForce,
                         LeafSystem, List, LogVectorOutput, MeshcatVisualizer,
                         ModelVisualizer, Parser, Simulator, SpatialForce,
                         StartMeshcat, Value)

from manipulation import FindResource, running_as_notebook

from pydrake.all import (AddMultibodyPlantSceneGraph, AngleAxis, BasicVector,
                         ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant,
                         MultibodyPositionToGeometryPose, Parser,
                         PiecewisePose, Quaternion, RigidTransform,
                         RollPitchYaw, RotationMatrix, SceneGraph, Simulator,
                         StartMeshcat, TrajectorySource)

import logging
from copy import copy
from enum import Enum
from IPython.display import SVG, display
import pydot

import numpy as np
from pydrake.all import (AbstractValue, AddMultibodyPlantSceneGraph, AngleAxis,
                         Concatenate, DiagramBuilder, InputPortIndex,
                         LeafSystem, MeshcatVisualizer, Parser,
                         PiecewisePolynomial, PiecewisePose, PointCloud,
                         PortSwitch, RandomGenerator, RigidTransform,
                         RollPitchYaw, Simulator, StartMeshcat,
                         UniformlyRandomRotationMatrix)

from manipulation import FindResource, running_as_notebook
from manipulation.clutter import GenerateAntipodalGraspCandidate
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.pick import (MakeGripperCommandTrajectory, MakeGripperFrames,
                               MakeGripperPoseTrajectory)
from manipulation.scenarios import (AddIiwaDifferentialIK, AddPackagePaths,
                                    MakeManipulationStation, ycb)
from pydrake.geometry import (
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Role,
    StartMeshcat,
)
from pydrake.multibody.meshcat import JointSliders
from manipulation.meshcat_utils import MeshcatPoseSliders, WsgButton

import os

class NoDiffIKWarnings(logging.Filter):
    def filter(self, record):
        return not record.getMessage().startswith('Differential IK')

logging.getLogger("drake").addFilter(NoDiffIKWarnings())

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://0cf2191d-16c4-4b77-b4c8-52b22020d459.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


# Rubik's Cube (2x2)

In [ ]:
cube_file = FindResource("models/rubiks_cube_2_by_2.sdf")

meshcat.Delete()

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(cube_file)

visualizer.Run(loop_once=not running_as_notebook)

meshcat.DeleteAddedControls()


Use Ctrl-C or click 'Stop Running' to quit


In [ ]:
rotate_about = 'x'
#rotate_about = 'y'
#rotate_about = 'z'

meshcat.Delete()
builder = DiagramBuilder()

plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.001)
Parser(plant).AddModels(cube_file)

plant.WeldFrames(plant.world_frame(),
                 plant.GetFrameByName("center"))
plant.Finalize()

visualizer = MeshcatVisualizer.AddToBuilder(
      builder, scene_graph, meshcat)
ContactVisualizer.AddToBuilder(builder, plant, meshcat)
#logger = LogVectorOutput(plant.get_state_output_port(), builder)

class CubePusher(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
        forces_cls = Value[List[ExternallyAppliedSpatialForce]]
        self.DeclareAbstractOutputPort(
            "applied_force",
            lambda: forces_cls(),
            self.CalcOutput)

    def CalcOutput(self, context, output):
        forces = []
        for x,y,z in product([0,1], repeat=3):
            force = ExternallyAppliedSpatialForce()
            force.body_index = plant.GetBodyByName(f"box_{x}_{y}_{z}").index()
            # shift from [0, 1] to [-1, 1]
            x = 2 * x - 1
            y = 2 * y - 1
            z = 2 * z - 1
            force.p_BoBq_B = -0.0125*np.array([x, y, z]) # world 0, 0, 0
            if rotate_about == 'x':
                force.F_Bq_W = SpatialForce(
                    tau=-0.2 * np.array([1 if x < 0 else -1, 0, 0]),
                    f=[0, 0, 0])
            elif rotate_about == 'y':
                force.F_Bq_W = SpatialForce(
                    tau=0.2 * np.array([0, 1 if y > 0 else -1, 0]),
                    f=[0, 0, 0])
            else:
                force.F_Bq_W = SpatialForce(
                    tau=0.2 * np.array([0, 0, 1 if z > 0 else -1]),
                    f=[0, 0, 0])
        forces.append(force)
        output.set_value(forces)

pusher = builder.AddSystem(CubePusher())
builder.Connect(pusher.get_output_port(),
                plant.get_applied_spatial_force_input_port())

diagram = builder.Build()

simulator = Simulator(diagram)
context = simulator.get_context()

if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(10)
else:
    simulator.AdvanceTo(0.1)


# Rubik's Cube (3x3)

In [ ]:

def generate_3_by_3():
    filename = FindResource('models/rubiks_cube.sdf')
    box_size = 0.025
    box_mass = 0.03
    inertia = box_mass*(box_size**2)/6
    inertia = inertia * 100  # scale inertia to help numerics

    with open(filename, 'w') as f:
        f.write("""
<?xml version="1.0"?>
<!-- Autogenerated by rubiks_cube.ipynb -->
<sdf version="1.7">
  <model name="rubiks_cube">
    <link name="center"/>
""")

        for x,y,z in product([-1,0,1], repeat=3):
            sx = x if x>=0 else "m1"
            sy = y if y>=0 else "m1"
            sz = z if z>=0 else "m1"
            suffix = f"_{sx}_{sy}_{sz}"
            x_color = [0, 0, 1] if x < 0 else [0, 1, 0]
            y_color = [1, 0.5, 0] if y < 0 else [1, 0, 0]
            z_color = [1, 1, 0] if z < 0 else [1, 1, 1]
            f.write(f"""
    <link name="box{suffix}">
      <pose>{box_size*x} {box_size*y} {box_size*z} 0 0 0</pose>
      <inertial>
        <mass>0.03</mass>
        <inertia>
          <ixx>0.0003125</ixx>
          <ixy>0.0</ixy>
          <ixz>0.0</ixz>
          <iyy>0.0003125</iyy>
          <iyz>0.0</iyz>
          <izz>0.0003125</izz>
        </inertia>
      </inertial>
      <collision name="collision">
        <geometry>
          <box>
            <size>{box_size} {box_size} {box_size}</size>
          </box>
        </geometry>
      </collision>
      <visual name="black">
        <geometry>
          <box>
            <size>{box_size} {box_size} {box_size}</size>
          </box>
        </geometry>
        <material>
          <diffuse>0 0 0 1</diffuse>
        </material>
      </visual>
      <visual name="x">
        <pose>{x*box_size/2} 0 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.0002 0.02 0.02</size>
          </box>
        </geometry>
        <material>
          <diffuse>{x_color[0]} {x_color[1]} {x_color[2]} 1</diffuse>
        </material>
      </visual>
      <visual name="y">
        <pose>0 {y*box_size/2} 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.02 0.0002 0.02</size>
          </box>
        </geometry>
        <material>
          <diffuse>{y_color[0]} {y_color[1]} {y_color[2]} 1</diffuse>
        </material>
      </visual>
      <visual name="z">
        <pose>0 0 {z*box_size/2} 0 0 0</pose>
        <geometry>
          <box>
            <size>0.02 0.02 0.0002</size>
          </box>
        </geometry>
        <material>
          <diffuse>{z_color[0]} {z_color[1]} {z_color[2]} 1</diffuse>
        </material>
      </visual>
    </link>      
    <joint name="ball{suffix}" type="ball">
        <pose>{-x*box_size} {-y*box_size} {-z*box_size} 0 0 0</pose> <!-- in child frame -->
        <parent>center</parent>
        <child>box{suffix}</child>
        <axis>
            <dynamics><damping>0.1</damping></dynamics>
            <limit><effort>0</effort></limit>
        </axis>
    </joint>
""")

        f.write("""
  </model>
</sdf>
""")

generate_3_by_3()

In [ ]:
cube_file = FindResource("models/rubiks_cube.sdf")

meshcat.Delete()

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(cube_file)

visualizer.Run(loop_once=not running_as_notebook)

meshcat.DeleteAddedControls()


Use Ctrl-C or click 'Stop Running' to quit


In [ ]:
model_directives = """
directives:
- add_model:
    name: iiwa0
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa0::iiwa_link_0
    X_PC:
        translation: [0, 0.7, 0]
        rotation: !Rpy { deg: [0, 0, -90]}
- add_model:
    name: wsg0
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa0::iiwa_link_7
    child: wsg0::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
# - add_model:
#     name: iiwa1
#     file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
#     default_joint_positions:
#         iiwa_joint_1: [-1.57]
#         iiwa_joint_2: [0.1]
#         iiwa_joint_3: [0]
#         iiwa_joint_4: [-1.2]
#         iiwa_joint_5: [0]
#         iiwa_joint_6: [ 1.6]
#         iiwa_joint_7: [0]
# - add_weld:
#     parent: world
#     child: iiwa1::iiwa_link_0
#     X_PC:
#         # translation: [0, -0.89, 0]
#         translation: [0, -0.88, 0]
#         rotation: !Rpy { deg: [0, 0, 180]}
# - add_model:
#     name: wsg1
#     file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
# - add_weld:
#     parent: iiwa1::iiwa_link_7
#     child: wsg1::body
#     X_PC:
#         translation: [0, 0, 0.09]
#         rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: floor
    file: package://manipulation/floor.sdf
- add_weld:
    parent: world
    child: floor::box
    X_PC:
        translation: [0, 0, -0.05]
- add_model:
    name: cube
    file: package://manipulation/rubiks_cube_2_by_2.sdf
    default_free_body_pose:
        center:
            translation: [0, 0, 0.5]
# - add_model:
#     name: mount
#     file: package://project/platform.sdf
# - add_weld:
#     parent: world
#     child: mount::bin_base
#     X_PC:
#         translation: [0, 0, 0]
# - add_model:
#     name: cube
#     file: package://project/temp_cube.sdf
#     default_free_body_pose:
#         base_link:
#             translation: [0, 0, 0]
# - add_weld:
#     parent: world
#     child: cube::center
#     X_PC:
#         translation: [0, 0, 0.04]
- add_model:
    name: mount
    file: package://project/bin.sdf
- add_weld:
    parent: world
    child: mount::bin_base
    X_PC:
        translation: [0, 0, 0]
# - add_model:
#     name: cube
#     file: package://project/temp_cube.sdf
#     default_free_body_pose:
#         base_link:
#             translation: [0, 0, 0]
"""

class IIWA_Painter():
    def __init__(self, traj=None):
        builder = DiagramBuilder()
        # set up the system of manipulation station
        self.station = builder.AddSystem(
            MakeManipulationStation(model_directives, package_xmls=["./package.xml"]))

        # builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")

        # optionally add trajectory source
        if traj is not None:
            traj_V_G = traj.MakeDerivative()
            V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
            self.controller = builder.AddSystem(
                PseudoInverseController(self.plant))
            builder.Connect(V_G_source.get_output_port(),
                            self.controller.GetInputPort("V_G"))

            self.integrator = builder.AddSystem(Integrator(7))
            builder.Connect(self.controller.get_output_port(),
                            self.integrator.get_input_port())
            builder.Connect(self.integrator.get_output_port(),
                            self.station.GetInputPort("iiwa0_position"))
            builder.Connect(
                self.station.GetOutputPort("iiwa0_position_measured"),
                self.controller.GetInputPort("iiwa0_position"))

        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat, params)

        wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(wsg_position.get_output_port(),
                        self.station.GetInputPort("wsg0_position"))

        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.world_frame = self.plant.world_frame()

        context = self.CreateDefaultContext()

        self.diagram.Publish(context)


    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies
        
        Input: 
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.
        
        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(meshcat, "painter/" + name,
                        length=length, radius=radius, X_PT=X_WF)

    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context)
        station_context = self.diagram.GetMutableSubsystemContext(
            self.station, context)

        # provide initial states
        q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                       -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("iiwa0")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg0")
        self.plant.SetPositions(plant_context, wsg, [-0.025, 0.025])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])        

        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)

        return context


    def get_X_WG(self, context=None):

        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
                    plant_context,
                    frame_A=self.world_frame,
                    frame_B=self.gripper_frame)
        return X_WG

    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)
        simulator.set_target_realtime_rate(1.0)

        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)

class PseudoInverseController(LeafSystem):
    """
    same controller seen in-class
    """
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa0")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_G", BasicVector(6))
        self.q_port = self.DeclareVectorInputPort("iiwa0_position", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa0_velocity", BasicVector(7),
                                     self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G) #important
        output.SetFromVector(v)


In [ ]:
# define center and radius
radius = 0.05
p0 = [0, 0.0, 0.06]
R0 = RotationMatrix(np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]]).T)
X_WorldCenter = RigidTransform(R0, p0)

num_key_frames = 10
"""
you may use different thetas as long as your trajectory starts
from the Start Frame above and your rotation is positive
in the world frame about +z axis
thetas = np.linspace(0, 2*np.pi, num_key_frames)
"""
thetas = np.linspace(0, 2*np.pi, num_key_frames)

painter = IIWA_Painter()

/tmp/ipykernel_76/3400793911.py:148: DrakeDeprecationWarning: (Deprecated) See ForcedPublish() 
*/ (Deprecated.)

Deprecated:
    Use ForcedPublish() instead This will be removed from Drake on or
    after 2023-03-01.
  self.diagram.Publish(context)


In [ ]:
X_WG = painter.get_X_WG() 
painter.visualize_frame('gripper_current', X_WG)

In [ ]:
def compose_circular_key_frames(thetas, X_WorldCenter, X_WorldGripper_init):
    """    
    returns: a list of RigidTransforms
    """
    ## this is an template, replace your code below
    key_frame_poses_in_world = [X_WorldGripper_init]
    for theta in thetas:

        ThetaTurn = RigidTransform(RotationMatrix.MakeYRotation(- theta))

        RadiusOutward = RigidTransform(RotationMatrix.MakeYRotation(0), [0, 0, radius])

        TangentToPathTurn = RigidTransform(RotationMatrix.MakeYRotation(np.pi))

        this_pose = RigidTransform(((X_WorldCenter @ ThetaTurn) @ RadiusOutward) @ TangentToPathTurn)

        key_frame_poses_in_world.append(this_pose)
        
    return key_frame_poses_in_world

In [ ]:
# check key frames instead of interpolated trajectory
def visualize_key_frames(frame_poses):
    for i, pose in enumerate(frame_poses):
        painter.visualize_frame('frame_{}'.format(i), pose, length=0.05)
        
key_frame_poses = compose_circular_key_frames(thetas, X_WorldCenter, painter.get_X_WG())   
visualize_key_frames(key_frame_poses)

In [ ]:
X_WorldGripper_init = painter.get_X_WG()
X_WorldGripper_init = RigidTransform(painter.get_X_WG().rotation(), [0, 0.26, 0.5]) 
total_time = 20
key_frame_poses = compose_circular_key_frames(thetas, X_WorldCenter, X_WorldGripper_init)  
times = np.linspace(0, total_time, num_key_frames+1)
traj = PiecewisePose.MakeLinear(times, key_frame_poses)

In [ ]:
painter = IIWA_Painter(traj)
painter.paint(sim_duration=total_time + 10)

/tmp/ipykernel_76/3400793911.py:148: DrakeDeprecationWarning: (Deprecated) See ForcedPublish() 
*/ (Deprecated.)

Deprecated:
    Use ForcedPublish() instead This will be removed from Drake on or
    after 2023-03-01.
  self.diagram.Publish(context)


In [ ]:
model_directives = """
directives:
- add_model:
    name: iiwa0
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa0::iiwa_link_0
    X_PC:
        translation: [0, 0.7, 0]
        rotation: !Rpy { deg: [0, 0, 0]}
- add_model:
    name: wsg0
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa0::iiwa_link_7
    child: wsg0::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: iiwa1
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1.57]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa1::iiwa_link_0
    X_PC:
        # translation: [0, -0.89, 0]
        translation: [0, -0.88, 0]
        rotation: !Rpy { deg: [0, 0, 180]}
- add_model:
    name: wsg1
    file: package://drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa1::iiwa_link_7
    child: wsg1::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy { deg: [90, 0, 90]}
- add_model:
    name: floor
    file: package://manipulation/floor.sdf
- add_weld:
    parent: world
    child: floor::box
    X_PC:
        translation: [0, 0, -0.05]
- add_model:
    name: cube
    file: package://manipulation/rubiks_cube_2_by_2.sdf
    default_free_body_pose:
        center:
            translation: [0, 0, 0.2]
#             rotation: !Rpy { deg: [0, 40, -90]}
# - add_model:
#     name: mount
#     file: package://project/platform.sdf
# - add_weld:
#     parent: world
#     child: mount::bin_base
#     X_PC:
#         translation: [0, 0, 0]
# - add_model:
#     name: cube
#     file: package://project/temp_cube.sdf
#     default_free_body_pose:
#         base_link:
#             translation: [0, 0, 0]
# - add_weld:
#     parent: world
#     child: cube::center
#     X_PC:
#         translation: [0, 0, 0.04]
# - add_model:
#     name: mount
#     file: package://project/bin.sdf
# - add_weld:
#     parent: world
#     child: mount::bin_base
#     X_PC:
#         translation: [0.0, 0, 0.01]
#         rotation: !Rpy { deg: [0, 45, 90]}
# - add_model:
#     name: cube
#     file: package://project/temp_cube.sdf
#     default_free_body_pose:
#         base_link:
#             translation: [0, 0, 0]
"""

# builder = DiagramBuilder()

# station = builder.AddSystem(
#     MakeManipulationStation(model_directives))
# plant = station.GetSubsystemByName("plant")
# controller_plant = station.GetSubsystemByName(
#     "iiwa0_controller").get_multibody_plant_for_control()

# visualizer = MeshcatVisualizer.AddToBuilder(
#     builder, station.GetOutputPort("query_object"), meshcat)

# diagram = builder.Build()

# display(SVG(
#     pydot.graph_from_dot_data(
#         diagram.GetGraphvizString(max_depth=1))[0].create_svg()))

# simulator = Simulator(diagram)
# context = simulator.get_mutable_context()
# station_context = station.GetMyMutableContextFromRoot(context)
# plant_context = plant.GetMyMutableContextFromRoot(context)

# q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('iiwa0'))
# station.GetInputPort('iiwa0_position').FixValue(station_context, q0)
# station.GetInputPort('iiwa1_position').FixValue(station_context, q0)

# station.GetInputPort('wsg0_position').FixValue(station_context, [0])
# station.GetInputPort('wsg1_position').FixValue(station_context, [0])

# Confirm that simulation works:
# simulator.AdvanceTo(0.1)
def add_cube(plant):
    cube_file = FindResource("models/rubiks_cube_2_by_2.sdf")
    Parser(plant).AddModels(cube_file)

def teleop_3d_bimanual():
    builder = DiagramBuilder()

    time_step = 0.001
    station = builder.AddSystem(
    MakeManipulationStation(model_directives, time_step = time_step, package_xmls=["./package.xml"]))
    plant = station.GetSubsystemByName("plant")
    controller_plant = station.GetSubsystemByName(
        "iiwa0_controller").get_multibody_plant_for_control()


    # Add a meshcat visualizer.
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, station.GetOutputPort("query_object"), meshcat)
    meshcat.ResetRenderMode()
    meshcat.DeleteAddedControls()

    # Set up differential inverse kinematics.
    differential_ik = AddIiwaDifferentialIK(
        builder,
        controller_plant,
        frame=controller_plant.GetFrameByName("iiwa_link_7"))
    builder.Connect(differential_ik.get_output_port(),
                    station.GetInputPort("iiwa0_position"))
    builder.Connect(station.GetOutputPort("iiwa0_state_estimated"),
                    differential_ik.GetInputPort("robot_state"))

    # Set up teleop widgets.
    teleop = builder.AddSystem(
        MeshcatPoseSliders(
            meshcat,
            min_range=MeshcatPoseSliders.MinRange(roll=0,
                                                  pitch=-0.5,
                                                  yaw=-np.pi,
                                                  x=-0.6,
                                                  y=-0.9,
                                                  z=0.0),
            max_range=MeshcatPoseSliders.MaxRange(roll=2 * np.pi,
                                                  pitch=np.pi,
                                                  yaw=np.pi,
                                                  x=0.8,
                                                  y=-0.3,
                                                  z=1.1),
            body_index=plant.GetBodyByName("iiwa_link_7", 
                plant.GetModelInstanceByName('iiwa0')).index()))
    builder.Connect(teleop.get_output_port(0),
                    differential_ik.get_input_port(0))
    builder.Connect(station.GetOutputPort("body_poses"),
                    teleop.GetInputPort("body_poses"))
    wsg_teleop = builder.AddSystem(WsgButton(meshcat))
    builder.Connect(wsg_teleop.get_output_port(0),
                    station.GetInputPort("wsg0_position"))




    diagram = builder.Build()
    simulator = Simulator(diagram)
    context = simulator.get_mutable_context()
    station_context = station.GetMyMutableContextFromRoot(context)
    plant_context = plant.GetMyMutableContextFromRoot(context)

    q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('iiwa1'))
    print(q0)
    # station.GetInputPort('iiwa0_position').FixValue(station_context, q0)
    # q0[5] = -np.pi/4
    # q0[1] = 1.385
    # station.GetInputPort('iiwa1_position').FixValue(station_context, q0)

    start = np.array(np.append(q0, 0.1))
    print(start.shape)
    above = np.array([-1.57,  1.285,   0,   0,   0,    -np.pi/4,   0,  0.1])
    down = np.array([-1.57,  1.385,   0,   0,   0,    -np.pi/4,   0,  0.1])
    grab = np.array([-1.57,  1.385,   0,   0,   0,    -np.pi/4,   0,  0])
    up = np.array([-1.57,  1.285,   0,   0,   0,    -np.pi/4,   0,  0])

    start_above = np.linspace(start, above, 5)
    above_down = np.linspace(above, down, 5)
    down_grab = np.linspace(down, grab, 5)
    grab_up = np.linspace(grab, up, 5)


    offset = 0
    keyframes = [
        [-1.57,  1.385,   0.5,   0,   0,    -np.pi/4,   0,  0.07],
        [-1.57,  1.385,   0,   -1.2,   0,    -np.pi/4,   0,  0.07],
        [-1.57,  1.385,   0,   -1.2,   0,    -np.pi/4 + offset,   0,  0.052],
        [-1.57,  1.385,   0,   -1.2,   0,    -np.pi/4 + offset,   0,  0.05],
        [-1.57,  1.385,   0,   -1.2,   0,    -np.pi/4 + offset,   0,  0.05],
        [-1.57,  1.385,   0,   -1.2,   0,    -np.pi/4 + offset,   0,  0.04],
        [-1.57,  1.385,   0,   -1.2,   0,    -np.pi/4 + offset,   0,  0.04],
        # [-1.57,  1.335,   0,   -1.2,   0,    -np.pi/4,   0,  0.00],
        # [-1.57,  1.285,   0,   -1.2,   0,    -np.pi/4,   0,  0.00],
    ]

    # keyframes = np.vstack((start_above, above_down, down_grab, grab_up))
    # print(q0)
    # q0[4] = 0
    # keyframes = [q0+[0.1]]
    # keyframes= np.linspace(start, up, 10)
    # keyframes = keyframes.tolist()
    # print(keyframes)
    # station.GetInputPort('wsg0_position').FixValue(station_context, [0])
    keyframes = []
    station.GetInputPort('wsg1_position').FixValue(station_context, [0])
    # station.GetInputPort('wsg0_position').FixValue(station_context, [.0515])

    if running_as_notebook:  # Then we're not just running as a test on CI.
        simulator.set_target_realtime_rate(1.0)
        meshcat.AddButton("Stop Simulation", "Escape")
        print("Press Escape to stop the simulation")
        while meshcat.GetButtonClicks("Stop Simulation") < 1:
            # q0[1]+=.1
            # station.GetInputPort('iiwa1_position').FixValue(station_context, q0)
            
            if (keyframes):
                frame = keyframes.pop(0)
                station.GetInputPort('iiwa1_position').FixValue(station_context, frame[:7])
                station.GetInputPort('wsg1_position').FixValue(station_context, frame[-1])
            simulator.AdvanceTo(simulator.get_context().get_time() + 2.0)
        meshcat.DeleteButton("Stop Simulation")

    else:
        simulator.AdvanceTo(0.1)


teleop_3d_bimanual()

Keyboard Controls:
roll : KeyQ / KeyE
pitch : KeyW / KeyS
yaw : KeyA / KeyD
x : KeyJ / KeyL
y : KeyI / KeyK
z : KeyO / KeyU
Press Space to open/close the gripper
[-1.57  0.1   0.   -1.2   0.    1.6   0.  ]
(8,)
Press Escape to stop the simulation


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=03fc1d85-661b-4243-b727-b4cf06229651' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>